# LAB: Utilizando Pandas con datos de oferta de departamentos en la CABA

## Introducción

El objetivo de este LAB es trabajar con los conceptos de agregación a través de `groupby` y `pivot tables` mencionadas en la sección teórica. 

Para eso usaremos un dataset elaborado por el Gobierno de la Ciudad de Buenos Aires que releva el 

> [p]recio de oferta de venta de terrenos de la Ciudad Autónoma de Buenos Aires, representado en puntos de valor del suelo correspondientes a cada parcela. Con el fin de analizar el precio de oferta, se toma como unidad de referencia el precio del metro cuadrado (m2). Se utiliza el precio en dólares, puesto que ésta es la moneda que se usa en el mercado como referencia para este tipo de transacciones. Otras variables consideradas son las dimensiones, ubicación geográfica, y la normativa del Código de Planeamiento Urbano que rige la zona en que el terreno analizado se localiza. 

Pueden descargar el dataset y otros elaborados para los años 2001-2015 desde este [link](
https://data.buenosaires.gob.ar/dataset/departamentos-en-venta). En el pdf adjunto pueden encontrar una descripción sucinta de las variables relevadas.

Importamos los paquetes necesarios y cargamos los datos.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

In [2]:
df = pd.read_csv("ED_deptos_venta_2014.csv",encoding = "latin1", sep=";")

In [3]:
# Veamos los tipos de las columnas...
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12724 entries, 1 to 12724
Data columns (total 17 columns):
CALLE         12724 non-null object
NUMERO        12724 non-null int64
M2            12724 non-null int64
DOLARES       12724 non-null int64
U_S_M2        12724 non-null float64
AMBIENTES     12724 non-null int64
ANTIGUEDAD    9942 non-null float64
ORIENT        12724 non-null object
BAULERA       12724 non-null object
COCHERA       12724 non-null object
BAÃ.OS        12724 non-null int64
LAVADERO      12724 non-null object
TERRAZA       12724 non-null object
BARRIO        12724 non-null object
COMUNA        12724 non-null int64
LON           12724 non-null float64
LAT           12724 non-null float64
dtypes: float64(4), int64(6), object(7)
memory usage: 1.7+ MB


In [4]:
# Revisamos que los datos se han importado correctamente

df.head()

,CALLE,NUMERO,M2,DOLARES,U_S_M2,AMBIENTES,ANTIGUEDAD,ORIENT,BAULERA,COCHERA,BAÃ.OS,LAVADERO,TERRAZA,BARRIO,COMUNA,LON,LAT
1,11 DE SEPTIEMBRE DE 1888,700,560,1270000,2267.9,5,20.0,FRENTE,Si,Si,4,No,Si,PALERMO,14,-58.439523,-34.570692
2,11 DE SEPTIEMBRE DE 1888,800,126,295000,2341.3,4,45.0,S/D,No,Si,0,No,No,PALERMO,14,-58.440256,-34.570313
3,11 DE SEPTIEMBRE DE 1888,900,109,320000,2935.8,4,20.0,FRENTE,Si,Si,3,Si,Si,PALERMO,14,-58.441321,-34.569487
4,11 DE SEPTIEMBRE DE 1888,900,150,400000,2666.7,4,37.0,FRENTE,Si,Si,2,Si,Si,PALERMO,14,-58.441321,-34.569487
5,11 DE SEPTIEMBRE DE 1888,900,270,950000,3518.5,4,30.0,FRENTE,No,No,3,No,Si,PALERMO,14,-58.441321,-34.569487


### 1. ¿Cuál es el valor promedio del metro cuadrado en la CABA?

In [5]:
df['M2'].mean()

86.2639107198994

### 2. ¿Cómo es la distribución cuartílica de la antigüedad de las viviendas de la CABA?

In [6]:
antig_qt = pd.qcut(df['ANTIGUEDAD'], 4)
df['ANTIGUEDAD'].quantile([.25,.5,.75])

0.25     5.0
0.50    30.0
0.75    40.0
Name: ANTIGUEDAD, dtype: float64

### 3. Calcular las distribuciones decílicas de la antigüedad y precios de la viviendas en CABA


In [63]:
antig_qt_dec = pd.qcut(df['ANTIGUEDAD'], [.1,.2,.3,.4,.5,.6,.7,.8,.9,1])
antig_dist_dol = df.pivot_table(['DOLARES', 'M2'], antig_qt_dec, aggfunc={'DOLARES':['mean', 'count'], 'M2':'sum'})
antig_dist_dol

DOLARES                     M2
                count           mean     sum
ANTIGUEDAD                                  
(-0.001, 3.0]    2030  170084.162069  123689
(3.0, 10.0]      1069  238858.043031   84463
(10.0, 25.0]     1093  294179.550778  120515
(25.0, 30.0]     1004  210770.995020   92117
(30.0, 35.0]     1023  180166.040078   84355
(35.0, 40.0]     1390  180789.561151  115611
(40.0, 45.0]      605  171877.917355   49120
(45.0, 55.0]      799  198628.468085   72858
(55.0, 115.0]     929  251695.238967  111840

### 4. ¿Cuál es la media de precio por metro cuadrado para cada barrio de la CABA? Ordenar los datos para indicar cuál es el barrio más caro.
Nota: realizar el cálculo tanto con groupby como con pivot tables

### 5. ¿En qué barrio hay una mayor dispersión de precios de metro cuadrado? Ordenar los valores para identificar el mayor.

(Recordar la fórmula del coeficiente de variación para medir la dispersión)

**Pistas**
* Una primera opción es definir una función con arrays y utilizar `.apply()`

* Una segunda, es generar dos series: 
  - una con el método `.std()` y dividirla por otra serie generada con `mean()`

**Bonus:** ¿cuál le parece mejor y por qué?

### 6. ¿En qué barrio los departamentos son más grandes? 
Nota: se puede realizar el cálculo tanto con groupby como con pivot tables

### 7. En general, puede observarse alguna diferencia entre el precio mediano de aquellos departamentos que tienen terraza y los que no? ¿Y en el tamaño?

### 8. Genere un `DataFrame` que agregue la información del precio por M2 (`U_S_M2`), ambientes (`AMBIENTES`) y antigüedad (`ANTIGUEDAD`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

### 9. Si tuvieran que tomar una decisión sobre dónde invertir en una casa... ¿qué barrio elegirían? ¿Por qué?